In [69]:
import pandas as pd
import numpy as np
import random
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## Blueprint:
Step 1: Read in the Data

Step 2: Make Count features (count & count cat), and score_differential

Step 3: Generate pitcher overall percentages, fill the NaNs for missing pitch types, and make pitch_type_cat feature

Step 4: Make Batter Scouting Report and Merge those features into the df (percentage faced, est_woba, est_babip, iso_value, and chase %, by pitch_type_category)

Step 5: Make a python Class for Pitcher (filter out a specific pitcher from the df)
Make Pitcher Scouting Report and Merge those Features into the df (pitch_type tendencies, overall and by count category)

Step 6: Make Game pitch count column, and make trailing pitch type features (L1, L5, L10): figure out strategy for 1st 5 and first 15 pitches of each game (maybe use overall tendencies, maybe use count tendencies, or maybe get historical first5/first 10 tendencies)

Step 7: Pitcher/Batter matchup history

Step 8: Write code that does step 1-8, iteratively, updating each month of 2018 and into the 2019 season (to prevent leakage of future information into the training set)

Step 9: Make list of features we will use for model input vector + (merge w/ umpire data if we can get it)

Step 10: Train Models
    
    
    
    
    
#potential other features to engineer in future: 

-either on base or not- replace the baserunner id w/ 1 or 0.. potentially down the road can use stolen base stats to categorize the baserunner into 2 or 3 tiers maybe (since if a speedy player is one first base, pitcher may be less likely to throw as many breaking balls for example).. not a huge priority feature tho

-under the trailing pitches section, add a feature for when the prev pitch:
    - was a base hit
    - was a home run
    - gave up a run (or runs): check whether the post_bat_score > bat_score

-pitcher feature for battting stats onballs in play:
    -est_woba/babip/isovalue/chase% for diff pitches in this game, prev game, prev 5      games, prev season, and/or historical 

## Step 1: Read in the Data

In [70]:
fname = 'pitches_2017.pkl'
df_17 = pd.read_pickle(fname, compression='zip').drop(columns=['pitcher.1', 'fielder_2.1'])

#convert the pitch type for UN (unknown) to np.nan
df_17['pitch_type'] = df_17['pitch_type'].replace({'UN':np.nan})

#fix some faulty data that has number of balls listed as 4:
df_17['balls'] = df_17['balls'].replace({4.0: 3.0})

In [71]:
df_17.head().T

,0,1,2,3,4
index,593,614,618,636,650
pitch_type,FC,FC,FF,CH,FT
game_date,2017-10-01 00:00:00,2017-10-01 00:00:00,2017-10-01 00:00:00,2017-10-01 00:00:00,2017-10-01 00:00:00
release_speed,91.9,91.8,94.7,82.8,95.9
release_pos_x,-1.1948,-1.0777,-0.6208,-0.9901,-0.8903
release_pos_z,5.9921,6.1772,6.3387,6.0626,6.1341
player_name,Roberto Osuna,Roberto Osuna,Roberto Osuna,Roberto Osuna,Roberto Osuna
batter,595885,595885,595885,595885,595885
pitcher,532077,532077,532077,532077,532077
events,field_out,NaN,NaN,NaN,NaN


## Step 2: Make Count features (count & count cat)

In [56]:
def make_count_features_and_score_diff(df):
    df = df.copy()
    
    #convert balls and strikes columns to strings (first to int)
    df['balls'] = df['balls'].astype('int').astype('str')
    df['strikes'] = df['strikes'].astype('int').astype('str')

    #create count feature:
    df['_count'] = df['balls'] + df['strikes']

    #create count category feature:
    count_map = {'00':'neutral', '21':'neutral', '32':'neutral', '10':'behind', '20':'behind',
             '30':'behind', '31':'behind', '01':'ahead', '02':'ahead', '11':'ahead',
             '12':'ahead', '22':'ahead'}
    df['count_cat'] = df['_count'].replace(count_map)
    
    #cant decide whether to drop the balls and strikes feature, keeping for now- check feature importances later
    #df = df.drop(columns=['balls', 'strikes'])
    
    #make score_differential feature
    df['score_diff'] = df['fld_score'] - df['bat_score']
    return df

df_17 = make_count_features_and_score_diff(df_17)

## Step 3: Generate pitcher overall percentages, fill the NaNs for missing pitch types, and make pitch_type_cat feature

In [24]:
def gen_pitcher_percentages(df):
    df = df.copy()
    #get all the unique pitcher names in the df
    pitcher_list = df.pitcher.unique().tolist()
    #initialize empty dictionary to store each pitcher and their pitches and percentages for each pitch
    pitcher_dict = {}
    #iterate over each pitcher:
    for pitcher in pitcher_list:
        #assign the normalized value_counts to a variable
        pitch_percentages = df[df.pitcher == pitcher].pitch_type.value_counts(normalize=True)
        #convert that Series object to a dict and assign it as the value to the pitcher dictionary 
        #(pitcher name as key)
        pitcher_dict[pitcher] = pitch_percentages.to_dict()
    return pitcher_dict

#generate the dictionary of pitch type % by pitcher
pitcher_dict = gen_pitcher_percentages(df_17)

def fill_pitch_type_nans(df, pitcher_dict):
    df = df.copy()
    #grab the rows where pitch_type is null:
    nulls = df[df.pitch_type.isna()]
    
    #iterate over each null row
    for index, row in nulls.iterrows():
        #use the % for that pitcher for each pitch type he throws to generate a random pitch type with 
        #that % as weight
        pitch = random.choices(population=list(pitcher_dict[row.pitcher].keys()), 
                               weights=list(pitcher_dict[row.pitcher].values()), 
                               k=1)[0]
        #fill the NaN value with the randomly generated pitch
        df.at[index, 'pitch_type'] = pitch
    return df

df_17 = fill_pitch_type_nans(df_17, pitcher_dict)

#create map for pitch type into categories:
pitch_type_map = {'FA':'fastball', 'FF':'fastball', 'FT':'fastball', 'FC':'fastball',
                  'FS':'fastball', 'SI':'fastball', 'SF':'fastball', 'SL':'breaking',
                  'CB':'breaking', 'CU':'breaking', 'SC':'breaking', 'KC':'breaking',
                  'CH':'offspeed', 'KN':'offspeed', 'EP':'offspeed', 'FO':'pitchout', 
                  'PO':'pitchout'}

#create pitch cateogory feature
df_17['pitch_cat'] = df_17['pitch_type']
df_17['pitch_cat'] = df_17['pitch_cat'].replace(pitch_type_map)

## Step 4: Make Batter Scouting Report and Merge those features into the df (percentage faced, est_woba, est_babip, iso_value, and chase %, by pitch_type_category)

In [25]:
# Fill in code from batter scouting report notebook

## Step 5: Make Pitcher Scouting Report and Merge those Features into the df (pitch_type tendencies, overall and by count category)

Choose a pitcher to model: Justin Verlander example:

In [26]:
pitcher_df = df_17[df_17.pitcher == 434378.0].sort_values(by = ['game_date', 'at_bat_number', 'pitch_number'])

In [27]:
def get_pitch_tendencies(pitcher_df):
    #assign the normalized value counts for this pitchers pitch types to a dictionary
    pitcher_tendencies_overall = pitcher_df['pitch_type'].value_counts(normalize=True).to_dict()
    
    #initialize empty dict for count categories tendencies
    pitcher_tendencies_by_count = {}
    
    #loop over each count category and get the pitchers tendencies and add to the dict
    for cat in pitcher_df['count_cat'].unique().tolist():
        subset = pitcher_df[pitcher_df['count_cat'] == cat]
        pitcher_tendencies_by_count[cat] = subset['pitch_type'].value_counts(normalize=True).to_dict()
    return pitcher_tendencies_overall, pitcher_tendencies_by_count

In [28]:
overall, by_count = get_pitch_tendencies(pitcher_df)

In [29]:
overall

{'FF': 0.5766071934296233,
 'SL': 0.21467006513735487,
 'CU': 0.1600113282356273,
 'CH': 0.041348060039648825,
 'FC': 0.003964882469555367,
 'FT': 0.0033984706881903144}

In [30]:
by_count['neutral']['FF']

0.6719269102990033

In [31]:
pitcher_df.head(2)

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,stand,p_throws,home_team,away_team,type,hit_location,bb_type,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,_count,count_cat,pitch_cat
717221,14433,FF,2017-04-04,94.0,-2.5011,6.3799,Justin Verlander,573135.0,434378.0,NaN,called_strike,6.0,NaN,R,R,CWS,DET,S,NaN,NaN,2017.0,-1.6380,1.3832,0.4384,2.4509,NaN,NaN,NaN,0.0,1.0,Bot,NaN,NaN,543510.0,170404_182235,10.9326,-135.9093,-7.2560,-22.2188,29.3508,-14.0833,3.25,1.44,NaN,NaN,NaN,92.448,2587.0,5.680,490102.0,408234.0,435079.0,592206.0,578428.0,457708.0,592444.0,543484.0,54.8186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,4-Seam Fastball,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Strategic,Standard,00,neutral,fastball
717220,14423,FF,2017-04-04,95.1,-2.3831,6.3130,Justin Verlander,573135.0,434378.0,NaN,ball,14.0,NaN,R,R,CWS,DET,B,NaN,NaN,2017.0,-1.4071,1.6660,1.8448,1.8521,NaN,NaN,NaN,0.0,1.0,Bot,NaN,NaN,543510.0,170404_182256,13.8935,-137.1570,-9.4298,-20.4625,29.7620,-9.7076,3.25,1.44,NaN,NaN,NaN,93.507,2534.0,5.795,490102.0,408234.0,435079.0,592206.0,578428.0,457708.0,592444.0,543484.0,54.7032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,4-Seam Fastball,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Standard,Standard,01,ahead,fastball


In [32]:
def make_tendency_features(pitcher_df, pitcher_tendencies_overall, pitcher_tendencies_by_count):
    df = pitcher_df.copy()
    pitch_types = df.pitch_type.value_counts().index.tolist()
    for pitch_type in pitch_types:
        overall_feature = 'overall_' + pitch_type + '_perc'
        count_cat_feature = 'count_cat_' + pitch_type + '_perc'
        
        def get_overall_perc(x):
            return pitcher_tendencies_overall[x]
        def get_by_count_perc(x):
            return pitcher_tendencies_by_count[x][pitch_type]
        
        df[overall_feature] = pitch_type
        df[overall_feature] = df[overall_feature].apply(get_overall_perc)
        df[count_cat_feature] = df['count_cat'].apply(get_by_count_perc)
    return df

In [33]:
make_tendency_features(pitcher_df, overall, by_count).head()

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,stand,p_throws,home_team,away_team,type,hit_location,bb_type,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,_count,count_cat,pitch_cat,overall_FF_perc,count_cat_FF_perc,overall_SL_perc,count_cat_SL_perc,overall_CU_perc,count_cat_CU_perc,overall_CH_perc,count_cat_CH_perc,overall_FC_perc,count_cat_FC_perc,overall_FT_perc,count_cat_FT_perc
717221,14433,FF,2017-04-04,94.0,-2.5011,6.3799,Justin Verlander,573135.0,434378.0,NaN,called_strike,6.0,NaN,R,R,CWS,DET,S,NaN,NaN,2017.0,-1.6380,1.3832,0.4384,2.4509,NaN,NaN,NaN,0.0,1.0,Bot,NaN,NaN,543510.0,170404_182235,10.9326,-135.9093,-7.2560,-22.2188,29.3508,-14.0833,3.25,1.44,NaN,NaN,NaN,92.448,2587.0,5.680,490102.0,408234.0,435079.0,592206.0,578428.0,457708.0,592444.0,543484.0,54.8186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,4-Seam Fastball,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Strategic,Standard,00,neutral,fastball,0.576607,0.671927,0.21467,0.172757,0.160011,0.117940,0.041348,0.033223,0.003965,0.001661,0.003398,0.002492
717220,14423,FF,2017-04-04,95.1,-2.3831,6.3130,Justin Verlander,573135.0,434378.0,NaN,ball,14.0,NaN,R,R,CWS,DET,B,NaN,NaN,2017.0,-1.4071,1.6660,1.8448,1.8521,NaN,NaN,NaN,0.0,1.0,Bot,NaN,NaN,543510.0,170404_182256,13.8935,-137.1570,-9.4298,-20.4625,29.7620,-9.7076,3.25,1.44,NaN,NaN,NaN,93.507,2534.0,5.795,490102.0,408234.0,435079.0,592206.0,578428.0,457708.0,592444.0,543484.0,54.7032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,2.0,4-Seam Fastball,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Standard,Standard,01,ahead,fastball,0.576607,0.490929,0.21467,0.249588,0.160011,0.213854,0.041348,0.038483,0.003965,0.003299,0.003398,0.003848
717219,14412,CU,2017-04-04,79.2,-2.6276,6.1709,Justin Verlander,573135.0,434378.0,single,hit_into_play_no_out,1.0,Tyler Saladino singles on a ground ball to lef...,R,R,CWS,DET,X,7.0,ground_ball,2017.0,0.4102,-0.9814,-0.3723,2.7628,NaN,NaN,NaN,0.0,1.0,Bot,66.06,135.05,543510.0,170404_182315,4.0855,-114.8706,1.0709,2.6878,21.7332,-40.9471,3.25,1.44,NaN,90.3,-17.0,77.067,2842.0,4.989,490102.0,408234.0,435079.0,592206.0,578428.0,457708.0,592444.0,543484.0,55.5096,0.163,0.164,0.9,1.0,1.0,0.0,2.0,5.0,3.0,Curveball,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Standard,Standard,11,ahead,breaking,0.576607,0.490929,0.21467,0.249588,0.160011,0.213854,0.041348,0.038483,0.003965,0.003299,0.003398,0.003848
717218,14402,SL,2017-04-04,86.8,-2.5506,6.3337,Justin Verlander,641313.0,434378.0,NaN,swinging_strike,3.0,NaN,R,R,CWS,DET,S,NaN,NaN,2017.0,-0.1480,0.6201,0.6356,2.9052,NaN,NaN,573135.0,0.0,1.0,Bot,NaN,NaN,543510.0,170404_182445,7.7677,-125.9456,-3.2419,-2.8956,21.9236,-25.3681,3.32,1.51,NaN,NaN,NaN,85.759,2573.0,5.454,490102.0,408234.0,435079.0,592206.0,578428.0,457708.0,592444.0,543484.0,55.0449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,1.0,Slider,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Standard,Standard,00,neutral,breaking,0.576607,0.671927,0.21467,0.172757,0.160011,0.117940,0.041348,0.033223,0.003965,0.001661,0.003398,0.002492
717217,14395,SL,2017-04-04,89.3,-2.5144,6.3712,Justin Verlander,641313.0,434378.0,NaN,foul,3.0,NaN,R,R,CWS,DET,S,NaN,NaN,2017.0,-0.2297,0.9230,0.5960,2.8399,NaN,NaN,573135.0,0.0,1.0,Bot,NaN,NaN,543510.0,170404_182522,7.9815,-129.5219,-4.4926,-4.0720,25.0701,-21.3858,3.32,1.51,130.0,75.3,13.0,88.084,2488.0,5.537,490102.0,408234.0,435079.0,592206.0,578428.0,457708.0,592444.0,543484.0,54.

In [34]:
#potential other pitcher features to engineer in future: 
#est_woba/babip/isovalue/chase% for diff pitches in this game, prev game, prev 5 games, prev season, and/or historical -

## Step 6: Make Game pitch count column, and make trailing pitch type features (L1, L5, L10): figure out strategy for 1st 5 and first 10 picthes of each game (maybe use overall tendencies, maybe use count tendencies, or maybe get historical first5/first 10 tendencies)

In [35]:
#horrible function name but whatever- still need to add code to get trailing 15 pitch percentages

def make_game_pitchcount_and_trailing_pitch_features(pitcher_df, pitcher_tendencies_overall):
    df = pitcher_df.copy()
    games = df['game_pk'].unique().tolist()
    
    #take the first game and make the pitch count feature
    new_df = df[df['game_pk'] == games[0]]
    new_df['pitch_count'] = range(1, new_df.shape[0] + 1)
    
    #make the L1_pitch type feature:
    new_df['L1_pitch_type'] = new_df['pitch_type'].shift(periods=1)
    
    #make the trailing 5 pitches:
    for index, row in new_df.iterrows():
        #fill NaNs for L1_pitch using same method as when pitch_type was missing
        if row['pitch_count'] == 1:
            random_pitch = random.choices(population=list(pitcher_tendencies_overall.keys()), 
                               weights=list(pitcher_tendencies_overall.values()), 
                               k=1)[0]
            new_df.at[index, 'L1_pitch_type'] = random_pitch
            
        #for the first 5 rows, use overall pitcher tendencies    
        if row['pitch_count'] < 6:
            #fill with overall tendencies
            for pitch in list(pitcher_tendencies_overall.keys()):
                feature = 'L5_' + pitch + '_perc'
                new_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100        
        else:
            current_pitch = new_df.at[index, 'pitch_count']
            #make a subset of the prev 5 pitches
            subset = new_df[(new_df['pitch_count'] > current_pitch - 6) & (new_df['pitch_count'] < current_pitch)]
            #grab the value count percentages for the last 5 pitches
            subset_percentages = subset['pitch_type'].value_counts(normalize=True).to_dict()
            
            #iterate over all possible pitch types this pitcher throws:
            for pitch in list(pitcher_tendencies_overall.keys()):
                feature = 'L5_' + pitch + '_perc'
                #if he has thrown that pitch type in last 5
                try:
                    new_df.at[index, feature] = subset_percentages[pitch] * 100
                #except for when he hasnt thrown that type in last 5
                except:
                    new_df.at[index, feature] = 0
    
    #iterate the same process for the rest of his games:
    for game in games[1:]:
        game_df = df[df['game_pk'] == game] #get df for that game only
        game_df['pitch_count'] = range(1, game_df.shape[0] + 1) #make the pitch count for the game
        game_df['L1_pitch_type'] = game_df['pitch_type'].shift(periods=1)
        
        
        
        #make the trailing 5 pitches:
        for index, row in game_df.iterrows():
            #fill NaNs for L1_pitch using same method as when pitch_type was missing
            if row['pitch_count'] == 1:
                random_pitch = random.choices(population=list(pitcher_tendencies_overall.keys()), 
                               weights=list(pitcher_tendencies_overall.values()), 
                               k=1)[0]
                game_df.at[index, 'L1_pitch_type'] = random_pitch
            
            if row['pitch_count'] < 6:
                #fill with overall tendencies
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100          
            else:
                current_pitch = game_df.at[index, 'pitch_count']
                subset = game_df[(game_df['pitch_count'] > current_pitch - 6) & (game_df['pitch_count'] < current_pitch)]
                subset_percentages = subset['pitch_type'].value_counts(normalize=True).to_dict()
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    try:
                        game_df.at[index, feature] = subset_percentages[pitch] * 100
                    except:
                        game_df.at[index, feature] = 0
        
        #concatenate that game w/ updated picth count and trailing pitches w/ prev games
        new_df = pd.concat([new_df, game_df]) #concat the game_df w/ the prev games
    return new_df

In [36]:
%%time
pitcher_df = make_game_pitchcount_and_trailing_pitch_features(pitcher_df, overall)

/home/jm/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/home/jm/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/jm/.local/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http:/

CPU times: user 28.1 s, sys: 41.4 ms, total: 28.1 s
Wall time: 28.1 s


In [40]:
pitcher_df.head(10).T

,717221,717220,717219,717218,717217,717216,717215,717214,717213,717212
index,14433,14423,14412,14402,14395,14386,14375,14366,14357,14352
pitch_type,FF,FF,CU,SL,SL,FF,FF,FF,FF,FF
game_date,2017-04-04 00:00:00,2017-04-04 00:00:00,2017-04-04 00:00:00,2017-04-04 00:00:00,2017-04-04 00:00:00,2017-04-04 00:00:00,2017-04-04 00:00:00,2017-04-04 00:00:00,2017-04-04 00:00:00,2017-04-04 00:00:00
release_speed,94,95.1,79.2,86.8,89.3,95.2,95.5,94.2,94.2,93.9
release_pos_x,-2.5011,-2.3831,-2.6276,-2.5506,-2.5144,-2.4037,-2.2776,-2.5742,-2.5407,-2.4642
release_pos_z,6.3799,6.313,6.1709,6.3337,6.3712,6.4671,6.3718,6.3366,6.3798,6.409
player_name,Justin Verlander,Justin Verlander,Justin Verlander,Justin Verlander,Justin Verlander,Justin Verlander,Justin Verlander,Justin Verlander,Justin Verlander,Justin Verlander
batter,573135,573135,573135,641313,641313,641313,641313,466320,466320,466320
pitcher,434378,434378,434378,434378,434378,434378,434378,434378,434378,434378
events,NaN,NaN,single,NaN,NaN,NaN,strikeout,NaN,NaN,NaN


## Step 7: Pitcher/Batter matchup history

In [38]:
#Todo

## Step 8: Write code that does step 1-8, iteratively, updating each month of 2018 and into the 2019 season (to prevent leakage of future information into the training set)

In [39]:
#Todo 